# Imaging Science Senior Project
### Jared Gregor (jmg2586@rit.edu)
The purpose of this Python Notebook is to provide a pipeline to allow for Blink modeling from eye tracking video. 

As video based eye trackers are becoming more common, eye tracking algorithms are becoming more complex to improve accuracy. Many of these algorithms utilize the pupil location to calculate other things such as gaze location. When a person makes a blink, the pupil is temporarily occluded, causing the algorithms to fail. Future algorithms may try to leverage the frames before a blink to accurately determine where the pupil will be after the blink. To do this, the blink needs to be quantified and modeled.

All of the helper functions that were written by Jared are found in BlinkDetection_Functions.py

In [1]:
import BlinkDetection_Functions

## Evaluate Dataset
It is useful to understand what the dataset you're working with is. For this project, I will be using the Gaze in Wild dataset. The function below makes a csv with the number of videos per trial for every person and every eye. The output csv is saved in 'data/outputs'. 

In [ ]:
# Get the number of videos per person per eye for all experiments in dataset
dataset = '../data/giw-blink/'
BlinkDetection_Functions.datasetStats(dataset) # Creates csv with number of videos in dataset

In the raw dataset, all of the left eyes are rotated 180 degrees. 

To stay consistant while labeling, it is important that all of the labeled images are oriented in the same direction. Let's roatate all of the videos 180 degrees to fix this issue. 

It may be a bit more efficiant if instead of rotating all of the frames in each video, we only rotate the frames that will be used for labeling. If DeepLabCut was already used to extract frames, the second function below will rotate only those images. 

Note: Only use one or the other!

In [ ]:
# Rotate the dataset videos 180 degrees
# Not Recommended
path_video = "../data/giw-blink/left/Person_1_Trial_1"
path_video = "J:/jmgre/Documents/Senior Project/data/giw-blink-unsorted/left"
BlinkDetection_Functions.rotateVideos(path_video)

In [ ]:
# Rotate the DLC extracted frames 180 degrees
path_frames = "../DeepLabCut/DLC_10_01_20/10-01-20-P4T1L-2020-10-01/labeled-data"
BlinkDetection_Functions.rotateFrames(path_frames)

We can also reorganize the data to combine all of the data into one place. This will give us a good variety of images to be labeled and will hopefully work well across people.

In [ ]:
path = 'J:/jmgre/Documents/Senior Project/data/giw-blink-unsorted'
BlinkDetection_Functions.mixData(path)

## Setup a DeepLabCut Project

https://github.com/DeepLabCut/DeepLabCut

DeepLabCut (DLC) is a toolbox for markerless pose estimation. In this project, we will be utilizing DLC to train a model that is capable of tracking the upper and lower eyelids in the GIW dataset. DLC has well documented pipelines on their website for how to train such a model.

In [1]:
import deeplabcut

DLC can be ran entirely though a GUI, or each step can be ran as a Python function. In this project, both options are available.

### Option 1: Run DLC entirely through the GUI

In [87]:
deeplabcut.launch_dlc()

Created "J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\videos"
Created "J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\labeled-data"
Created "J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\training-datasets"
Created "J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\dlc-models"
Copying the videos
J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\videos\Person_1_Trial_1_Eye_0_Blink_2.avi
J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\videos\Person_1_Trial_2_Eye_0_Blink_14.avi
J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\videos\Person_2_Trial_1_Eye_1_Blink_43.avi
J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\videos\Person_2_Trial_2_Eye_1_Blink_56.avi
J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\videos\Person_2_Trial_3_Eye_0_Blink_8.avi
J:\jmgre\Docu

TypeError: expected str, bytes or os.PathLike object, not NoneType

### Option 2: Create new DLC project

In [ ]:
task = 'blink' # Enter the name of your experiment Task
experimenter = 'p1e0t1' # Enter the name of the experimenter
video = ["/../data/giw-blink/right/Person_1_Trial_1/"] # Enter the path of the folder to grab frames from

path_config_file = deeplabcut.create_new_project(task,experimenter,video,copy_videos=True,working_directory='../DeepLabCut') 

### Option 3: Open an existing DLC project

In [2]:
path_config_file = "/../DeepLabCut/blink-p1e0t1-2020-11-19/config.yaml" # Enter the path to the project YAML file
path_config_file = "J:/jmgre/Documents/Senior Project/DeepLabCut/BlinkModeling-GIW-2020-11-20/config.yaml"

## Edit the config.yaml file
Now with the project created, the config.yaml file needs to be edited. In this file define the number of frames to be extracted, body part labels, etc. The body part labels are the points that get selected for labeling. In this project, body part labels will be all the points on the upper and lower eyelids to be selected. 

More information about our specific labeling scheme can be found here: 
https://docs.google.com/document/d/1dg3mrVQUD0TRQfZFkRGLiroeJcudWhvhZFnq21o9jFs/edit?usp=sharing


## Extract Frames from videos
Frames need to be extracted from the dataset videos to be labeled. DLC's default frame selection is to use kmeans. 

DeepLabCut offers several options as to how these frames will be selected. To see these options, add a '?' after the function.

deeplabcut.extract_frames?

In [3]:
%matplotlib inline

deeplabcut.extract_frames(path_config_file)

nkModeling-GIW-2020-11-20\videos\Person_6_Trial_1_Eye_1_Blink_2.avi ?
46it [00:00, 344.20it/s]Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 0.15  seconds.
Extracting and downsampling... 46  frames from the video.
Kmeans clustering ... (this might take a while)

Do you want to extract (perhaps additional) frames for video: J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\videos\Person_6_Trial_2_Eye_0_Blink_2.avi ?
40it [00:00, 353.34it/s]Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 0.13  seconds.
Extracting and downsampling... 40  frames from the video.
Kmeans clustering ... (this might take a while)

Do you want to extract (perhaps additional) frames for video: J:\jmgre\Documents\Senior Project\DeepLabCut\BlinkModeling-GIW-2020-11-20\videos\Person_6_Trial_2_Eye_1_Blink_2.avi ?
40it [00:00, 354.93it/s]Extracting frames based on kmeans ...
Kmea

## Label frames
Once successfully extracted, each of the frames needs to be labeled. For this project, I will be labeling the upper eyelid, lower eyelid, and pupil. The labeling scheme is defined in the config.yaml file. Both eyelids will have 15 markers labeled left to right. The markers should line up along the edge of the eyeball and the eyelid. The pupil will be labeled with markers in each of the cardinal directions.

In [5]:
%gui wx

deeplabcut.label_frames(path_config_file)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


### Check the labels
It is worth checking the labels before training to be sure everything goes properly. The DLC check_labels function checks to be sure the labels were created and stored properly.

In [ ]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

In [ ]:
### Create Training dataset


In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Start training:
deeplabcut.train_network?

In [ ]:
deeplabcut.train_network(path_config_file)

## Evaluate network:
Now that we have trained our network, it is time to evaluate it's performance. DLC has a built in function for evaluating the network. 

deeplabcut.evaluate_network?

There are also a few optional steps we can take here including extracting outliers, refining the labels, and retraining the network. These steps should be referenced by the DLC documentation.

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

## Analyze new video
Now that we have a model that we are happy with, we can use it to analyze new eye tracking videos.

The results are stored in the same path as the input video.

In [ ]:
videofile_path = ['videos/video3.avi','videos/video4.avi'] #Enter a folder OR a list of videos to analyze.

deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.avi')

## Create labeled video (optional)
To illustrate the results of our model on the new video, DLC provides a function to produce videos with the markers overlaid. There are a ton of options they give us for creating these videos!

In [ ]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Fit polynomial to eyelids
DeepLabCut will output a spreadsheet with each bodypart found and it's x and y coordinates. Using these points, we can fit a polynomial to each of the eyelids in each frame.

In [ ]:
import csv



## Calculate Blink Statistics



### Percent closure / Blink amplitude



### Blink frequency (requires time)

### Blink Duration (requires time)

## Output for RIT-eyes
What will we actually be giving to RIT eyes so that the eyelids can be moving in blender similar to how we describe the shape in the video?